In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
d[:logpop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_1 = "
data{
    int total_tools[10];
    real logpop[10];
    int society[10];
}
parameters{
    real a;
    real bp;
    vector[10] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[10] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:10 ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.1",  model=m12_6_1, num_warmup=2000,
num_samples=2000, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_1_data = Dict("total_tools" => d[:total_tools], "logpop" => d[:logpop], "society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_1_data, ProjDir, diagnostics=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan' at line 15)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.


Inference for Stan model: m12_6_1_model
4 chains: each with iter=(2000,2000,2000,2000); warmup=(0,0,0,0); thin=(1,1,1,1); 8000 iterations saved.

Warmup took (0.98, 0.97, 0.94, 0.98) seconds, 3.9 seconds total
Sampling took (1.00, 1.1, 1.0, 1.1) seconds, 4.2 seconds total

                    Mean     MCSE   StdDev        5%       50%       95%    N_Eff  N_Eff/s    R_hat
lp__             9.3e+02  1.4e-01  3.4e+00   9.2e+02   9.3e+02   9.3e+02  5.6e+02  1.3e+02  1.0e+00
accept_stat__   

Describe the draws

In [6]:
describe(chn)

Iterations = 1:2000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 2000

Empirical Posterior Estimates:
                   Mean            SD         Naive SE        MCSE         ESS   
         lp__  927.977447875  3.4281564286 0.03832795406 0.12191005562  790.75547
accept_stat__    0.939004511  0.0866305771 0.00096855930 0.00191284149 2000.00000
   stepsize__    0.048379200  0.0037257265 0.00004165489 0.00041915078   79.00988
  treedepth__    5.901000000  0.4711440832 0.00526755099 0.01587689987  880.59471
 n_leapfrog__   73.373750000 33.6632360687 0.37636642096 0.80696647098 1740.20633
  divergent__    0.000125000  0.0111803399 0.00012500000 0.00012500000 2000.00000
     energy__ -921.457912625  4.2642237808 0.04767547123 0.13411257382 1010.97631
            a    1.087184609  0.7444713801 0.00832344307 0.01422523707 2000.00000
           bp    0.262354774  0.0804773193 0.00089976378 0.00152282005 2000.00000
  a_society.1   -0.201493528  0.2448623890 0.00273764473 0.00433

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*